### doc_entity
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E ENTIDADES NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.  

AS ENTIDADES A SEREM TRABALHADAS SÃO:
* PAÍSES
* PESSOAS

In [1]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import re

import pymysql
import getpass
from sshtunnel import SSHTunnelForwarder

from IPython.display import clear_output

import csv
import xlrd
import xlwt

Especifica qual sistema operacional está sendo usado. Renato = Linux ; Marcelo = nt (Windows) e define variáveis a partir disso

In [2]:
if os.name == 'nt':
    encoding_type = 'utf-8'
    ssh_priv_key = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa'
    ssh_user='marcelobribeiro'
    sql_user='marcelobribeiro'
    path_inputs = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/"
    #path_inputs = "C:/Users/MARCELO/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/"
    path_outputs = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/outputs/"
else:
    encoding_type = 'ISO-8859-1'
    ssh_priv_key = '/home/rsouza/.ssh/id_rsa'
    ssh_user='rsouza'
    sql_user='rsouza'
    path_inputs = "/home/rsouza/Documentos/text-learning-tools/inputs/"
    path_outputs = "/home/rsouza/Documentos/text-learning-tools/outputs/"

## Pessoas
Trabalhos de visualização com entidades pessoas

In [23]:
count = 0
aas_graph = []

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.146', 22),
                        ssh_private_key=ssh_priv_key,
                        ssh_private_key_password=pass_sshkey,
                        ssh_username=ssh_user,
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user=sql_user,
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute('''SELECT person_doc.person_id, person_doc.doc_id, persons.person_name FROM CPDOC_AS.person_doc 
    JOIN CPDOC_AS.persons
    ON person_doc.person_id = persons.id
    WHERE doc_id 
    IN (SELECT doc_id FROM person_doc GROUP BY doc_id 
    HAVING count(*) > 1)
    ORDER BY doc_id ASC  
    ''')  
    person_doc_data = cur.fetchall()
    
    cur.execute("SELECT * FROM CPDOC_AS.persons")
    persons_data = cur.fetchall()
    
    old_doc_id = ""
    co_ocurring_names = ""
    for row in person_doc_data:
        if row is None: break

        person_id = row[0]
        doc_id = row[1]
        person_name = '"'+row[2]+'"'
        
        if doc_id != old_doc_id: 
            aas_graph.append(co_ocurring_names)
            co_ocurring_names = person_name
        else:
            co_ocurring_names = co_ocurring_names+","+person_name
        old_doc_id = doc_id
aas_graph.remove("")

········
········


In [24]:
aas_graph[:20]

['"pedro ii","vieira souto"',
 '"vieira souto","francisco eduardo de paula machado"',
 '"vieira de mello","vieira souto"',
 '"vieira souto","rui barbosa"',
 '"paulo cabral","vieira souto"',
 '"vieira souto","pedro ernesto"',
 '"vieira souto","borges de medeiros"',
 '"vieira souto","haroldo valadao"',
 '"carlos chagas","vieira souto"',
 '"emilio garrastazu medici","costa e silva"',
 '"rodrigo otávio","arnaldo sussekind"',
 '"condessa pereira carneiro","pereira carneiro"',
 '"pires do rio","sapena pastor","almeida santos","arnaldo carrilho","augusto pinochet","bittencourt de sá","josé guilherme merquior","yolanda prada","carlos garcia","papa paulo vi"',
 '"jorge de carvalho e silva","carvalho pinto"',
 '"eurico ","eurico penteado"',
 '"macedo soares","mar territorial","marcelo caetano","paulo nogueira batista","roberto campos","albuquerque lima","bilac pinto","celso lafer","costa e silva","joão goulart","janio quadros"',
 '"eurico ","fernando costa","francisco campos","getulio vargas","g

In [25]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
aas_graph_file = path_outputs+"aas_persons_graph.csv"
with open(aas_graph_file, 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=",", quoting=csv.QUOTE_NONE, quotechar='', escapechar=' ')
    for i in range(len(aas_graph)):
        writer.writerow([aas_graph[i]])

## Países
Trabalhos de visualização com entidades países

Graph structure

In [107]:
count = 0
aas_graph = []

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.146', 22),
                        ssh_private_key=ssh_priv_key,
                        ssh_private_key_password=pass_sshkey,
                        ssh_username=ssh_user,
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user=sql_user,
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    Reescrever a seleção de dados. Priorizei a estrutura do tableau
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute('''SELECT country_doc.country_id, countries.name, COUNT(doc_id) AS doc_count
    FROM CPDOC_AS.country_doc 
    JOIN CPDOC_AS.countries
    ON country_doc.country_id = countries.id
    WHERE doc_id 
    IN (SELECT doc_id FROM country_doc GROUP BY doc_id 
    HAVING count(*) > 1)
    GROUP BY name
    ORDER BY name ASC
    ''')  
    country_doc_data = cur.fetchall()
    
    cur.execute("SELECT * FROM CPDOC_AS.countries")
    countrys_data = cur.fetchall()
    
    old_doc_id = ""
    co_ocurring_names = ""
    for row in country_doc_data:
        if row is None: break

        country_id = row[0]
        doc_id = row[1]
        country_name = '"'+row[2]+'"'
        
        if doc_id != old_doc_id: 
            aas_graph.append(co_ocurring_names)
            co_ocurring_names = country_name
        else:
            co_ocurring_names = co_ocurring_names+","+country_name
        old_doc_id = doc_id
aas_graph.remove("")

········
········


In [110]:
co_ocurring_names

'"mozambique","poland","germany","cuba","mali"'

Tableau structure

In [22]:
count = 0
workbook = xlwt.Workbook()
worksheet = workbook.add_sheet('country_doc')
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.146', 22),
                        ssh_private_key=ssh_priv_key,
                        ssh_private_key_password=pass_sshkey,
                        ssh_username=ssh_user,
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user=sql_user,
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    Reescrever a seleção de dados. Priorizei a estrutura do tableau
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute('''SELECT country_doc.country_id, countries.name, COUNT(doc_id) AS doc_count
    FROM CPDOC_AS.country_doc 
    JOIN CPDOC_AS.countries
    ON country_doc.country_id = countries.id
    WHERE doc_id 
    IN (SELECT doc_id FROM country_doc GROUP BY doc_id 
    HAVING count(*) > 1)
    GROUP BY name
    ORDER BY name ASC
    ''')  
    country_doc_data = cur.fetchall()
    
    worksheet.write(0, 0, 'country_id')
    worksheet.write(0, 1, 'country')
    worksheet.write(0, 2, 'country_freq')
    
    for row in country_doc_data:
        #if row is None: break

        country_id = row[0]
        country_name = row[1]
        country_freq = row[2]
        row_index = country_doc_data.index(row)+1
        
        
        worksheet.write(row_index, 0, country_id)
        worksheet.write(row_index, 1, country_name)
        worksheet.write(row_index, 2, country_freq)
workbook.save(path_outputs+'country_doc.xls')

········
········
